Let's see if we can solve a simple math problem with gpt-3.5

In [2]:
# Add prereq code to set open API key

import os
os.environ["OPENAI_API_KEY"] = "your-openai-key"

In [3]:


from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 500)

# Note: example taken from
# https://browse.arxiv.org/pdf/2303.12712.pdf

instruction = """
Andy harvests all the tomatoes from 18 plants that have 7 tomatoes each. If he dries half the
tomatoes and turns a third of the remainder into marinara sauce, how many tomatoes are left?
"""
print("Original Answer: ")
print(llm(instruction))

/opt/venv/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/venv/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Original Answer: 
First, we need to find out how many tomatoes Andy harvested in total:
18 plants * 7 tomatoes per plant = 126 tomatoes

Next, we need to find out how many tomatoes Andy dried:
126 tomatoes / 2 = 63 dried tomatoes

Now, we need to find out how many tomatoes are left after drying:
126 tomatoes - 63 dried tomatoes = 63 tomatoes remaining

Next, we need to find out how many tomatoes Andy turned into marinara sauce:
63 tomatoes / 3 = 21 tomatoes turned into marinara sauce

Finally, we need to find out how many tomatoes are left after making marinara sauce:
63 tomatoes - 21 tomatoes = 42 tomatoes left

Therefore, Andy has 42 tomatoes left after drying half of them and turning a third of the remainder into marinara sauce.


If we run this, we see an answer:
Original Answer: 
Andy harvests a total of 18 plants * 7 tomatoes/plant = <<18*7=126>>126 tomatoes.
He dries half of the tomatoes, so he has 126 tomatoes / 2 = <<126/2=63>>63 tomatoes left.
He turns a third of the remaining tomatoes into marinara sauce, so he has 63 tomatoes * 1/3 = <<63*1/3=21>>21 tomatoes left. Answer: \boxed{21}.

In reality, Andy will have 63 - 21 = 42 tomatoes left

Let's see how we can do this with Chain Of Thought Reasoning

First, let's setup a few examples of how we want LLM to think about the problem

In [4]:

question1 = """
Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?
"""
answer1 = """
    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears
"""
question2 = """
Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?
"""
answer2 = """
    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.
    half of the remaining ones are turned into jam, or 300 * 1/2 = 150. Therefore, Sergei is left with 300 - 150, or 150 pears
"""

Next, let's setup a prompt template for one question/answer pair, and put our examples into an array

In [5]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="{question}\n{answer}")
# TODO: setup an array of question / answer examples
examples = [
    {
        "question": question1,
        "answer": answer1
    },
    {
        "question": question2,
        "answer": answer2
    }
]

Next, let's setup the FewShotPromptTemplate

In [6]:
# TODO: setup chain of thought prompt template based on few shot prompt template
cot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt = example_prompt,
    suffix="Use these questions and answers to give correct response to the problem below: {input}",
    input_variables = ['input']
    
)

Finally, let's use the same instruction as before and see if we get a different answer

In [7]:

cot_text = cot_prompt.format(input=instruction)
print("=== Chain of Thought Prompt ===")
print(cot_text)
print("=== Chain of Thought Answer ===")
print(llm(cot_text))

=== Chain of Thought Prompt ===

Karen harvests all the pears from 20 trees that have 10 pears each. She  throws a third of them away as they are rotten,
and turns a quarter of the remaining ones into jam. How many are left?


    First, let's calculate how many pears Gloria harvests: it's 20 * 10 = 200. 
    Then, let's calculate how many are rotten: 200 * 1/3 = 66.
    Thus, we know how many are left after she throws a third of them away: 200 - 66 = 134.
    1/4 of the remaining ones are turned into jam, or 134 * 1/4 = 33. Therefore, Karen is left with 134 - 33, or 101 pears



Sergei harvests all the strawberries from 50 plants that have 8 stawberries each. He freezes a quarter of them,
and turns half of the remaining ones into jam. How many are left?


    First, let's calculate how many strawberries Sergei harvests: it's 50 * 8 = 400. 
    Then, let's calculate how many are frozen: 400 * 1/4 = 100.
    Thus, we know how many are left after he freezes 100 of them: 400 - 100 = 300.
